## Access to market / Gambia

One of the most important aspects in development is access to/from resources. For example access to  hospitals, to schools, to markets...

The idea of this project is to focus on measuring this accessibility, either by assessing the transportation road conditions (find new, missing roads, or pavement quality) or calculate the distance from the harvested fields to the ports or markets.

For example, in Gambia [1/3 of the GDP](https://en.wikipedia.org/wiki/Economy_of_the_Gambia) is agriculture, and about [75% of the population depends on crops](https://rainforests.mongabay.com/deforestation/archive/Gambia.htm).

We could, for example, calculate first the location of planted areas, and then the travel times between these and the closest villages, or port (for exports). This will give us information of the operating cost and effort to produce the harvest, and could help us calculate the impact when a particular road is upgraded, or degraded.

### Data available:

- Landsat/Copernicus Level-1 at different times.
- Satellogic Hyperspectral where available.
- Historical Hyperion where available.


## This notebook

This notebook summarizes the idea of the project, links to relevant sources and kickstarts the initial steps getting the software and data to start working


### Troubleshoot
* This is meant to work with python 3.
* If you see `Import error` it will probably work by doing `conda install <package>`
* IF you get an error `Import error: /Users/brunosan/anaconda3/lib/libgdal.20.dylib. Reason: image not found` try `conda install  libgdal=2.0.0`

## Load a satellogic hyperspectral data from the region


Hyperspectral data can easily overpower your system resources (31 bands x 1500 width x 1500 height x 16bit/pixel is a lot or memory).

The strategy here is going to be to:
* Use the RGB color raster created by Satellogic, 
* Downsample the RGB 25% (25%^2 or ~6% of the filesize) to make it easier to work with, 
* Select a small windows of interest, and 
* Finally load all the hyperspectral data for that window.

Once you know the exact computation you might want to script it for the whole raster.

# How to get the Satellogic hypersectral data

You should have received a user/password (if not, email brunosan@satellogic.com). Once in the data portal (Telluric), [this links will go straight to the search in the Catalog for Gambia](https://telluric.satellogic.com/catalog/shared/N4IgRghgzgpgthADiAXCAFgF04qKD0+A1gPYBOJAdhAHQDmMJdNArpQJYC06M7AJjAA2YGGWYD8mdoJhR8FCHyh18ADwC8wVQF8AZAE9N+vQC9NJ7SAA0IAMYxKmUagDanAIwBOAOyeADN4ArDSeAEzeAMx+EQAsAGyeEVbugYF+Mak0oRERoWmBEQAccQC6NrYsZGQOmADK9pQwAJIAIqggjQDuUBCYEQD6CLYU-RUi-XneMIV+gaG2cXyeMWDeoYoQfquFhe6Rvu4AZhD9fv0x-RHWdoLstkQwfPUOzW1oXT19gxDDJKMs40m01m80Wy1W6z4m22u32niOJzOFyuNnQJAAbqJHs9Gq1UJQWIJBDYEOxKAAFCCNQQAYUq1UcABUIGB2tUoITMFBrodpE4yAAlEgsJzclAuFwgQ4kEg4MhkzAgMqSjmIRC3ZzKkCICh8Fi2TDUOAwJVWSVSY1QTAQODILVQBowfr8U0qx2wTDOviukCfIS3Jz9I0mkplEAARxYon0qFA0tlOoV7WAAB0QJh9IgYGmUGnySRBPo6FQ01Y07YZWQ+GTerIcxK3O5CjQInEZnEm7t0pEAskYjQYn44oPPIF3LMYqFPMVlR5m63253xzEe95kqEB1FQnFQn5EnFW4FZ+4N-5Zv5QqFB35CstQuvN35t7v94fj6e-Oen1fP7fJ32ByHEcxwnKcZzNOcWzbIcl27KI13cftB2HPcQMCSdp1KUNLBsP0iXYQNg3aa4HReL1iNw90YE9F00BIlg1Q1MgKKlCg4H6C0TTomxMD+TiWNuOACNQcdPxsEhDkOD1UD8ElqI2a1Y20GwdRIPUDSItAxhNZSQBMGU4FQOIaG8GJChyCzLIsmw+DICBOlEWprUwFgxRAEgs0ab0eI8nFXnaD5egGIYRm0iZAimGY5gWJYVjWDYtm8HY9giA5jlOc5Lmuch2DoGtBD8vF3hgbogu+X5-kBCLgWisE4shaEkthVL4XSpEstw8gpEoOgADF2CEJRXFACgRS4lwwykTAZCc+UepYw5CUESlMHQFiHhjbj3KrZw6OUkbhScVxJoImbMDmugFqWla1q2jaWPIARmL2qwDrG46eNOmBZrJS6tsWokbvWmBNpAbKduesHtBKbQgA).

From there, select an image. Below we chose [`257e8052c6d94b72ada0b788173791fa`](https://telluric.satellogic.com/catalog/scene/newsat3_macro_cube_257e8052c6d94b72ada0b788173791fa_0_4_3)

Notice: A whole scene set weights roughly *3Gb* compressed.

![](telluric_search.png)

## Download the data

In [ ]:
#Authenticate on telluric
import requests
url = 'https://auth.telluric.satellogic.com/api-token-auth/'
payload = {'username':'socialimpact','password':'sclmpct2018'}

print("Getting token...")
r = requests.post(url, data=payload)
if r.status_code != 200:
    raise ValueError("Telluric response error: %s" %r.text) 
telluric_token="JWT "+r.json()['token']
print(telluric_token[0:10]+"*masked*")

In [ ]:
#get scene id
import json

# https://telluric.satellogic.com/docs/#operation/list
url = 'https://telluric.satellogic.com/v2/scenes/'

footprint ={
        "type": "Point",
        "coordinates": [
          -16.64703369140625,
          13.447059093856021
        ]
      }

payload = {'footprint': json.dumps(footprint),
           'productname':'cube'}
header = { 'authorization' : telluric_token}
print("Requesting scene...")
r = requests.get(url, params=payload, headers=header)
if r.status_code != 200:
    raise ValueError("Telluric response error: %s" %r.text) 
response=r.json()
scene_id=response['results'][0]['scene_id']
print(scene_id)

In [ ]:
response['results'][0]['rasters'][:3]  # we can retrieve information about individual bands

In [ ]:
#download specific hyperspectral band to a file (<30 s with a good connection)
raster_827 = response['results'][0]['rasters'][1]

url = raster_827['url']
filename = raster_827['file_name']
header = { 'authorization' : telluric_token}

# http://docs.python-requests.org/en/master/user/quickstart/#raw-response-content
r = requests.get(url, headers=header, stream=True)
if r.status_code != 200:
    raise ValueError("Telluric response error: %s" %r.text) 

with open(filename, 'wb') as fd:
    for chunk in r.iter_content(chunk_size=128):
        fd.write(chunk)

In [ ]:
#get download id for the whole raster
url = 'https://telluric.satellogic.com/v2/scenes/download/'

header = { 'authorization' : telluric_token}
data = {'scene_id':scene_id,
        'async': 1}  # Important! This will prepare the download in the background for us
print("Requesting download...")
r = requests.get(url, params=data, headers=header)
if r.status_code != 200:
    raise ValueError("Telluric response error: %s" %r.text) 
response = r.json()
response

In [ ]:
#after a while, the download is ready 
#run this cell to check the download progress and wait until it is 100
requests.get(r.json()['status_path'], headers=header).json()

In [ ]:
#download raster to a file (<10 minutes with a good connection)
url = response['download_url']
filename = response['filename']
header = { 'authorization' : telluric_token}

# http://docs.python-requests.org/en/master/user/quickstart/#raw-response-content
r = requests.get(url, headers=header, stream=True)
if r.status_code != 200:
    raise ValueError("Telluric response error: %s" %r.text) 

with open(filename, 'wb') as fd:
    for chunk in r.iter_content(chunk_size=128):
        fd.write(chunk)

In [ ]:
#unzip the contents
import os
from zipfile import ZipFile

data_path = "data/satellogic/macro"
os.makedirs(data_path)

with ZipFile(filename, 'r') as fp:
    fp.extractall(data_path)

## Load the data

In [ ]:
import os
from glob import glob

hypercube_folder="./data/satellogic/macro/newsat3_macro_cube_257e8052c6d94b72ada0b788173791fa_0_4_3/rasters/"
files = [os.path.basename(fname) for fname in glob(hypercube_folder + "*.tif")]

rgb_file=[x for x in files if 'rgb_enhanced' in x][0]
hfiles=sorted(x for x in files if x[-6:]=='nm.tif')

print("RGB file: %s. Number of Spectral bands: %i" % (rgb_file,len(hfiles)))
print(files)

In [ ]:
import telluric as tl

path = hypercube_folder + rgb_file

rgb = tl.GeoRaster2.open(path)
rgb

In [ ]:
from pprint import pprint

stats = {
    'min': rgb.min(),
    'mean': rgb.mean(),
    'median': rgb.reduce('median'),
    'max': rgb.max()
}
print("File: ",path)
pprint(stats)

In [ ]:
print("Data types:", rgb.dtype)
print("Bounds:", rgb.bounds())
print("CRS:", rgb.crs)

In [ ]:
from rasterio import enums

rgb_resampled = rgb.resize(
    ratio=0.25,  # 1/4 on the side, typically 1/4^2 reduction is filesize
    resampling=enums.Resampling.nearest
)
rgb_resampled.save(hypercube_folder + 'resampled_rgb2.tif')

In [ ]:
#quick overall plot
%matplotlib inline
import matplotlib.pyplot as plt

from rasterio.plot import show
from rasterio.plot import show_hist

fig, (axrgb, axhist) = plt.subplots(1, 2, figsize=(14,7))

show(rgb_resampled.image, ax=axrgb)
show_hist(rgb_resampled.image, bins=50, histtype='stepfilled',
      lw=0.0, stacked=False, alpha=0.3, ax=axhist)

In [ ]:
#Plot the subregion to focus on, around the capital of Gambia, Banjul
roi = tl.FileCollection.open("banjul.geojson").envelope

cropped = tl.GeoRaster2.open(path).crop(roi)
cropped.save(hypercube_folder + "banjul2.tif")

In [ ]:
fig, (axrgb, axhist) = plt.subplots(1, 2, figsize=(14,7))

show(cropped.image, ax=axrgb)
show_hist(cropped.image, bins=50, histtype='stepfilled',
      lw=0.0, stacked=False, alpha=0.3, ax=axhist)

In [ ]:
from telluric.georaster import merge_all

print("Reading files...", end='')
bands = []
for band in hfiles:
    print(band, end=", ")
    rs = tl.GeoRaster2.open(hypercube_folder + band)
    bands.append(rs)

cube_raster = (
    merge_all(bands, roi.reproject(rgb.crs))  # The reproject will determine the final CRS
    #.astype(np.uint8)  # Cast to uint8 to save space  
)
print("")

In [ ]:
cube_raster

In [ ]:
cube = cube_raster.astype(np.uint8).image.transpose(1, 2, 0)

## Visualize the data

In [ ]:
%matplotlib notebook

from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def macro_plot(c):
    wavelengths = np.asarray([450, 462, 475, 488, 502, 516, 530, 550, 570, 582, 595, 608, 616, 670, 680, 690, 700, 710, 720, 730, 740, 750, 
                                   760 , 770, 780, 790, 800, 810, 820, 827])

    i=int(c.shape[2]/2)    
    x=int(c.shape[0]/2)
    y=int(c.shape[1]/2)
    
    fig=plt.figure(0)
    im=plt.subplot(121)
    s=plt.subplot(122)

    im.cla()
    im.imshow(c[:,:,i])

    red=[620,750]
    green=[495,570]
    blue=[450,495]
    def spectra(i,x,y):
        s.cla()
        s.plot(wavelengths,c[x,y,:]/255,'o-')
        s.axvline(x=wavelengths[i],linestyle=':',color='black')
        s.set_title('Pixel intensity')
        s.set_ylim(0,1)
    
    #add RGB reference
        for p in [
        patches.Rectangle(
            (red[0], 0), red[1]-red[0], 1,
            alpha=.1,Color='red'
        ),
        patches.Rectangle(
            (green[0], 0), green[1]-green[0], 1,
            alpha=.1,Color='green'
        ),
        patches.Rectangle(
            (blue[0], 0), blue[1]-blue[0], 1,
            alpha=.1,Color='blue'
        ),
    ]:
            s.add_patch(p)
        plt.show()

    def onclick(event):
        y=int(event.xdata)
        x=int(event.ydata)
        i=i_slider.value
        spectra(i,x,y)
    im.figure.canvas.mpl_connect('button_press_event', onclick)
    

    def spectrogram(i):
        im.imshow(c[:,:,i], cmap='gray')
        spectra(i,x,y)

    spectrogram(i)
    i_slider = widgets.IntSlider(min=0,
                             max=c.shape[2]-1,
                             step=1,
                             value=c.shape[2]/2,
                             description='Channel')
    interact(spectrogram, i=i_slider)

macro_plot(cube)

In [ ]:
##Check for image shifts (i vs i+1). Some images could have inter-channel spatial drift
##If found shift to align

from skimage.feature import register_translation

for i in np.arange(len(hfiles[:])-1):
    shift, error, diffphase = register_translation(cube[920:1020,320:420,15], cube[920:1020,320:420,i],1)
    print(i,shift)
print("")


# Land classifier

The following section points into ways one can classify  a scene without ground data.
For more information check out http://www.spectralpython.net/algorithms.html

uses a simple k-means

In [ ]:
from spectral import *
k=30
loops=10
(m, c) = kmeans(cube, k, loops)

In [ ]:
import matplotlib.cm as cm
km=plt.figure(4)

base = plt.cm.get_cmap(cm.jet)
cmap=base.from_list('', base(np.linspace(0, 1, k)), k)

plt.imshow(m,cmap=cmap)
plt.colorbar

In [ ]:
import pylab
f=pylab.figure()
f.hold(1)
for i in range(c.shape[0]):
    pylab.plot(c[i])
    
pylab.show()

In [ ]:
#PCA
#When doing principal components, the more components, the less information, hence they decay

pc = principal_components(cube)

#COV shows the correlation among wavelengths, which, as you can see is highest in the Infrared  (>20)
#(most probably for the amount of water in the scene), which has low reflectance in that range.

v = imshow(pc.cov)

In [ ]:
#Helper function to plot 3D cubes
def spectral_plot(c):
 
    i=int(c.shape[2]/2)    
    x=int(c.shape[0]/2)
    y=int(c.shape[1]/2)
    
    fig=plt.figure(2)
    im=plt.subplot(121)
    s=plt.subplot(122)

    im.cla()
    im.imshow(c[:,:,i])

    def spectra(i,x,y):
        s.cla()
        s.plot(c[x,y,:]/255,'o-')
        s.axvline(x=i,linestyle=':',color='black')
        s.set_title('Pixel intensity')
        s.set_ylim(0,1)
    

    def onclick(event):
        y=int(event.xdata)
        x=int(event.ydata)
        i=i_slider.value
        spectra(i,x,y)
    im.figure.canvas.mpl_connect('button_press_event', onclick)
    

    def spectrogram(i):
        im.imshow(c[:,:,i], cmap='gray')
        spectra(i,x,y)

    spectrogram(i)
    i_slider = widgets.IntSlider(min=0,
                             max=c.shape[2]-1,
                             step=1,
                             value=c.shape[2]/2,
                             description='Channel')
    interact(spectrogram, i=i_slider)


In [ ]:
fraction=0.999

pc_0999 = pc.reduce(fraction=fraction)

# How many eigenvalues are left?

print("Reducing with PCA to %2.1f%% of original image variance, yields %i frames."%(fraction*100,len(pc_0999.eigenvalues)))

img_pc = pc_0999.transform(cube)



spectral_plot(img_pc)